In [74]:
import pandas as pd

# Load the CSV file
file_path = '/Users/aronharsfalvi/dev/projects/forex-trading-tools/backtest/data/eurusd/download/eurusd-tick-2023-01-23-2024-01-25.csv'
df = pd.read_csv(file_path)

# Convert timestamp to datetime (milliseconds since epoch)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Assume bidPrice is for sell orders and askPrice is for buy orders
df.set_index('timestamp', inplace=True)

df.head()

,askPrice,bidPrice
timestamp,,
2023-01-23 00:00:00.129,1.08644,1.08639
2023-01-23 00:00:00.231,1.08647,1.08643
2023-01-23 00:00:01.226,1.08648,1.08645
2023-01-23 00:00:01.328,1.08651,1.08646
2023-01-23 00:00:01.430,1.08651,1.08647


In [75]:
import numpy as np

def simulate_trades(df, sl_pip):
    trades = []  # Store trade outcomes ('win' or 'lose')
    trade_details = []  # Detailed info about each trade
    in_position = False
    entry_price = None
    current_direction = "short"
    is_first_position = True
    STOP_LOSS = sl_pip * PIP
    TAKE_PROFIT = sl_pip * 3 * PIP

    for index, row in df.iterrows():
        # Check trading time
        # if LONDON_OPEN <= index.time() <= LONDON_CLOSE:
        if is_first_position == True:
            # Randomly decide to open a long or short position if not already in one
            entry_price = row['askPrice'] if current_direction == 'long' else row['bidPrice']
            in_position = True
            entry_time = index
            is_first_position = False
        elif in_position and not is_first_position :
            # Check for stop loss and take profit conditions
            if current_direction == 'long':
                if row['bidPrice'] >= entry_price + TAKE_PROFIT or row['bidPrice'] <= entry_price - STOP_LOSS:
                    in_position = False
            else:  # current_direction == 'short'
                if row['askPrice'] <= entry_price - TAKE_PROFIT or row['askPrice'] >= entry_price + STOP_LOSS:
                    in_position = False
        elif not in_position and not is_first_position: 
            # Determine trade outcome
            outcome = 'win' if (
                (current_direction == 'long' and row['bidPrice'] >= entry_price) or
                (current_direction == 'short' and row['askPrice'] <= entry_price)
            ) else 'lose'
            trades.append(outcome)

            # Record trade details
            trade_details.append({
                'entry_time': entry_time,
                'entry_price': entry_price,
                'exit_price': row['bidPrice'] if current_direction == 'long' else row['askPrice'],
                'direction': current_direction,
                'outcome': outcome
            })
            entry_price = row['askPrice'] if current_direction == 'long' else row['bidPrice']
            in_position = True
            entry_time = index

            # Flip direction on loss
            if outcome == 'lose':
                current_direction = 'long' if current_direction == 'short' else 'short'

    return trades, trade_details


def evaluate_performance(trades):
    wins = trades.count('win')
    losses = trades.count('lose')
    total_trades = wins + losses
    win_ratio = wins / total_trades if total_trades > 0 else 0

    # Assuming risk-free rate is 0 for simplicity. Adjust as needed.
    returns = np.array([1 if trade == 'win' else -1 for trade in trades])
    sharpe_ratio = np.mean(returns) / np.std(returns) if np.std(returns) != 0 else 0

    return win_ratio, sharpe_ratio, wins, losses

LONDON_OPEN = pd.to_datetime('08:00:00', format='%H:%M:%S').time()
LONDON_CLOSE = pd.to_datetime('16:00:00', format='%H:%M:%S').time()
PIP = 0.0001

In [76]:
import numpy as np

# Constants



# After simulating trades
trades, trade_details = simulate_trades(df, 5)
win_ratio, sharpe_ratio, wins, losses = evaluate_performance(trades)

print(f"SL pip size: {5}, Win Ratio: {win_ratio:.2f}, Sharpe Ratio: {sharpe_ratio:.2f}, Wins: {wins}, Losses: {losses}")



SL pip size: 5, Win Ratio: 0.25, Sharpe Ratio: -0.59, Wins: 1949, Losses: 5932


In [77]:
def create_csv_from_trades(trade_details, filename='/Users/aronharsfalvi/Downloads/trade_details.csv'):
    """
    Creates a CSV file from the trade_details data.

    :param trade_details: List of dictionaries with trade details.
    :param filename: Name of the CSV file to create.
    """
    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(trade_details)

    # Convert the 'entry_time' and 'exit_time' to a readable string format if needed
    # This step is optional and can be customized based on your preference
    df['entry_time'] = df['entry_time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
    
    # Export the DataFrame to a CSV file
    df.to_csv(filename, index=False)
    print(f"CSV file '{filename}' has been created.")

create_csv_from_trades(trade_details)   


CSV file '/Users/aronharsfalvi/Downloads/trade_details.csv' has been created.
